# Генерация .wav файлов (мелодий)

## Создание функции для вычисления частоты нот

Частоты музыкальных нот следуют логарифмической шкале, известной как двенадцатитоновый темперированный строй. Основная идея заключается в том, что каждая следующая нота имеет частоту, которая на фиксированное отношение выше частоты предыдущей ноты. Это отношение составляет корень двенадцатой степени из двух (≈1.059463), так как октава делится на 12 равных частей.

Основная формула для вычисления частоты ноты:
$$
f(n) = f_0 \times (2^{1/12})^n
$$

где:

- $(f(n))$ - частота n-й ноты,
- $(f_0)$ - частота опорной ноты (например, A4 = 440 Hz),
- $(a)$ - корень двенадцатой степени из двух (\( a = 2^{1/12} \)),
- $(n)$ - количество полутонов между опорной нотой и искомой нотой (может быть отрицательным).

**Функция, вычисляющая частоту ноты:**

In [353]:
def note_frequency(base_freq, step):
    a = 2 ** (1 / 12)
    return base_freq * (a ** step)

In [354]:
note_frequency(440, 2)

493.8833012561241

#### ![](frequencies.jpeg)


**Создание списка и массива с частотами нот:**

В список добавлены ноты малой, первой и второй октав (3,4,5)

In [355]:
notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
frequencies = {}

def note_frequencies(base_note='A4', base_frequency=440.0, octaves=3):
    for octave_offset in range(-1, 2):
        for i, note in enumerate(notes):
            n = i - notes.index(base_note[:-1]) + octave_offset * 12
            frequency = note_frequency(base_frequency, n)
            octave = int(base_note[-1]) + octave_offset
            note_name = f"{note}{octave}"
            frequencies[note_name] = round(frequency, 2)
    return frequencies
    
note_frequencies = note_frequencies()
frequencies_array = list(note_frequencies.values())

In [356]:
note_frequencies

{'C3': 130.81,
 'C#3': 138.59,
 'D3': 146.83,
 'D#3': 155.56,
 'E3': 164.81,
 'F3': 174.61,
 'F#3': 185.0,
 'G3': 196.0,
 'G#3': 207.65,
 'A3': 220.0,
 'A#3': 233.08,
 'B3': 246.94,
 'C4': 261.63,
 'C#4': 277.18,
 'D4': 293.66,
 'D#4': 311.13,
 'E4': 329.63,
 'F4': 349.23,
 'F#4': 369.99,
 'G4': 392.0,
 'G#4': 415.3,
 'A4': 440.0,
 'A#4': 466.16,
 'B4': 493.88,
 'C5': 523.25,
 'C#5': 554.37,
 'D5': 587.33,
 'D#5': 622.25,
 'E5': 659.26,
 'F5': 698.46,
 'F#5': 739.99,
 'G5': 783.99,
 'G#5': 830.61,
 'A5': 880.0,
 'A#5': 932.33,
 'B5': 987.77}

In [357]:
frequencies_array

[130.81,
 138.59,
 146.83,
 155.56,
 164.81,
 174.61,
 185.0,
 196.0,
 207.65,
 220.0,
 233.08,
 246.94,
 261.63,
 277.18,
 293.66,
 311.13,
 329.63,
 349.23,
 369.99,
 392.0,
 415.3,
 440.0,
 466.16,
 493.88,
 523.25,
 554.37,
 587.33,
 622.25,
 659.26,
 698.46,
 739.99,
 783.99,
 830.61,
 880.0,
 932.33,
 987.77]

## Функция, воспроизводящая ноту

Следующая функция создает звуковую волну для заданной частоты ноты, где:
1. note_frequency - частота ноты
2. duration - длительность звука в секундах
3. sample_rate - частота дискретизации (44100 - по умолчанию)
4. Генерация синусоидальной волны происходит с помощью частоты ноты  и массива времени, формула определяет осцилляции синусоиды с частотой.

   Формула: $\text{wave} = \sin(2 \pi f t)\$
  
5. Волна нормализуется, чтобы ее амплитуда находилась в диапазоне от -32767 до 32767, соответствующем диапазону значений 16-битного аудио. Для этого волна умножается на $(2**15 - 1)$ и делится на максимальное абсолютное значение сигнала.
6. Для сохранения в формате .wav волна преобразуется в целочисленный формат 16 бит.

In [358]:
import numpy as np
from scipy.io.wavfile import write
import IPython.display as ipd
import IPython

In [359]:
def generate_wave(note_frequency, duration=1, sample_rate=44100):
    t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
    wave = np.sin(2 * np.pi * note_frequency * t)
    wave = wave * (2**15 - 1) / np.max(np.abs(wave))
    wave = wave.astype(np.int16)
    write('note.wav', sample_rate, wave)

In [360]:
generate_wave(440)
ipd.Audio('note.wav')

## Функция, воспроизводящая аккорд

**Функции, создающие мажорные и минорные трезвучия:**

Первая функция генерирует синусоидальную волну с заданной частотой и длительностью, создает звук используя основные ноты аккордов. 
По отдельности генерируются волны каждой из частот и суммируются для получения комбинированного звука.

В отличии от **generate_wave**, функция **sine_wave** генерирует только волну, используя еще один аргумент - амплитуду, для лучшего звучания при суммировании звуков. Амплитуда отвечает за громкость.

Частоты для нот аккорда выбираются исходя из заданной тональности.

Мажорные и минорные трезвучия состоят из трех ступеней основной тональности: 
- Тоника (1 ступень)
- Медианта (3 ступень)
- Доминанта (5 ступень)

В зависимости от мажора/минора, расстояние до 3 ступени отличается на полтона.

Функции сначала вычисляют частоту тоники, затем на ее основе строят аккорд.

![](Tone.png)

In [361]:
def sine_wave(frequency, duration, sample_rate=44100, amplitude=0.5):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

def major_triad(tone= 'A4', duration=1.0, sample_rate=44100):

    tone_index = notes.index(tone[:-1])
    octave = int(tone[-1])
    freq_key = f"{tone[:-1]}{octave}"
    tone_freq = note_frequencies[freq_key]

    mediant_freq = note_frequency(tone_freq, 4)
    dominant_freq = note_frequency(tone_freq, 7)

    frequencies = [tone_freq, mediant_freq, dominant_freq]

    sine_waves = [sine_wave(freq, duration, sample_rate) for freq in frequencies]
    
    wave = np.sum(sine_waves, axis=0)
    wave /= np.max(np.abs(wave))
    audio_wave = np.int16(wave * 32767)
    write('chord_maj.wav', sample_rate, audio_wave)
    return audio_wave


def minor_triad(tone= 'A4', duration=1.0, sample_rate=44100):

    tone_index = notes.index(tone[:-1])
    octave = int(tone[-1])
    freq_key = f"{tone[:-1]}{octave}"
    tone_freq = note_frequencies[freq_key]

    mediant_freq = note_frequency(tone_freq, 3)
    dominant_freq = note_frequency(tone_freq, 7)

    frequencies = [tone_freq, mediant_freq, dominant_freq]

    sine_waves = [sine_wave(freq, duration, sample_rate) for freq in frequencies]
    
    wave = np.sum(sine_waves, axis=0)
    wave /= np.max(np.abs(wave))
    audio_wave = np.int16(wave * 32767)
    write('chord_min.wav', sample_rate, audio_wave)
    return audio_wave

In [362]:
major_triad('F4')
ipd.Audio('chord_maj.wav')

## Объединение нот в мелодии, аккордов в цепочки и комбинирование мелодии с аккордами

**Функция, генерирующая рандомную цепочку аккордов:**

За основу взята первая ступень тональности (тоника), для гармоничного звучания последним аккордом будет доминанта, которая должна разрешаться обратно в тонику. В промежутке аккорды генерируются рандомно. 

In [363]:
import random
def chords_gen():
    second, third = random.sample([3, 4, 6, 7], 2)
    chords = [1, second, third, 5]
    return chords
chords_gen()

[1, 7, 4, 5]

Помимо ступеней, между нотами есть еще и расстояния, которые отличаются. Ниже созданы списки с номером ступени и расстоянием до нее от тоники. Расстояние измеряется в полутонах. 

In [364]:
    major_intervals = [0, 2, 4, 5, 7, 9, 11]
    minor_intervals = [0, 2, 3, 5, 7, 8, 10]

Ниже приведена функция, которая генерирует массив с частотами из рандомной цепочки аккордов и заданой тональности.

In [365]:
def chords_freq(tone= 'A3', key = 'minor', chords = [1, 6, 4, 5]):

    tone_index = notes.index(tone[:-1])
    octave = int(tone[-1])
    freq_key = f"{tone[:-1]}{octave}"
    tonic = note_frequencies[freq_key]
    dominant = note_frequency(tonic, 7)

    second = chords[1] - 1
    third = chords[2] - 1

    if key == 'major':

        second = major_intervals[second]
        third =  major_intervals[third]

    elif key == 'minor':
        second = minor_intervals[second]
        third =  minor_intervals[third]       
        
        
    second = note_frequency(tonic, second)
    third =  note_frequency(tonic, third)  
    
    frequencies = [tonic, second, third, dominant]
    
    return frequencies
    

In [366]:
chain = chords_gen()
chords_freq('E3', 'minor', chain)

[164.81, 293.6580354734188, 219.99495636576341, 246.93598934004595]

**Функция, создающая звуковую волну из массива с частотами:**

(Пока функция только для минорной тональности)

In [367]:
import numpy as np

In [368]:
def sine_wave(frequency, duration, sample_rate=44100, amplitude=0.5):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave
    
def major_triad(tone_freq= '440.0', duration=1.0, sample_rate=44100):

    mediant_freq = note_frequency(tone_freq, 4)
    dominant_freq = note_frequency(tone_freq, 7)

    frequencies = [tone_freq, mediant_freq, dominant_freq]

    sine_waves = [sine_wave(freq, duration, sample_rate) for freq in frequencies]
    
    wave = np.sum(sine_waves, axis=0)
    wave /= np.max(np.abs(wave))
    audio_wave = np.int16(wave * 32767)
    return audio_wave


def minor_triad(tone_freq= '440.0', duration=1.0, sample_rate=44100):

    mediant_freq = note_frequency(tone_freq, 3)
    dominant_freq = note_frequency(tone_freq, 7)

    frequencies = [tone_freq, mediant_freq, dominant_freq]

    sine_waves = [sine_wave(freq, duration, sample_rate) for freq in frequencies]
    
    wave = np.sum(sine_waves, axis=0)
    wave /= np.max(np.abs(wave))
    audio_wave = np.int16(wave * 32767)
    return audio_wave
    

In [374]:
def create_chord_wave(tone='A3', duration=4.0, key='minor', chain = [1, 6, 4, 5]):
    i = len(chain)
    chain = chords_gen()
    chords = chords_freq(tone, key, chain)

    
    tone = note_frequencies['A3']
    chord_1 = minor_triad(tone, duration/i)

    
    if chain[1] == 4:
        chord_2 = minor_triad(chords[1], duration/i)
    else:
        chord_2 = major_triad(chords[1], duration/i)

    
    if chain[2] == 4:
        chord_3 = minor_triad(chords[2], duration/i)
    else:
        chord_3 = major_triad(chords[2], duration/i)

    
    chord_4 = major_triad(chords[3], duration/i)

    wave = np.concatenate((chord_1, chord_2, chord_3, chord_4))
    sample_rate = 44100

    write('chord_chain.wav', sample_rate, wave)
    
    return wave


In [402]:
create_chord_wave(tone='G3')
ipd.Audio('chord_chain.wav')

In [1]:
def adjust_frequencies(frequencies):
    tonic = frequencies[0]
    
    def adjust_to_tonic(value, tonic):
        best_value = value
        smallest_distance = abs(value - tonic)
        
        # Try to find the closest value to tonic by dividing or multiplying by 2
        divided_value = value
        multiplied_value = value
        
        while divided_value >= tonic or multiplied_value <= tonic * 2:
            divided_value /= 2
            multiplied_value *= 2
            
            for val in [divided_value, multiplied_value]:
                distance = abs(val - tonic)
                if distance < smallest_distance:
                    best_value = val
                    smallest_distance = distance
                    
        return best_value
    
    for i in range(1, len(frequencies)):
        frequencies[i] = adjust_to_tonic(frequencies[i], tonic)
    
    return frequencies

# Пример использования
frequencies = [164.81, 293.6580354734188, 219.99495636576341, 246.93598934004595]
adjusted_frequencies = adjust_frequencies(frequencies)
print(adjusted_frequencies)


[164.81, 146.8290177367094, 109.99747818288171, 123.46799467002297]
